In [1]:
import cobra
from cobra.io import load_model
import escher
from escher import Builder
from time import sleep
import pandas
from cobra import Model, Reaction, Metabolite

In [2]:
#define model
model = load_model("iJO1366")
#Run FBA, see growth rate under normal import circumstances
solution = model.optimize()
print('Growth rate: ' + str(solution.objective_value) + ' 1/h')

Growth rate: 0.9823718127269633 1/h


In [ ]:
#what if we decrease 'amount' of glucose in medium (i.e. decrease import rate of glucose? what happens to growth rate?)
model.reactions.get_by_id("EX_glc__D_e").upper_bound = 1000

model.reactions.get_by_id("EX_glc__D_e").lower_bound = -10
solution = model.optimize()
print('Growth rate: ' + str(solution.objective_value) + ' 1/h')


In [ ]:
#convert fluxes to df
print('Growth rate: ' + str(solution.objective_value) + ' 1/h')
df = pandas.DataFrame.from_dict([solution.fluxes]).T
df.to_csv('FBA_max_biomass.csv')

In [ ]:
escher.list_available_maps()

In [9]:
builder = Builder(
    map_name='e_coli_core.Core metabolism',
    model_name='iJO1366',
)
builder

Builder()

In [ ]:
#simulate deletions
from cobra.flux_analysis import (
    single_gene_deletion, single_reaction_deletion, double_gene_deletion,
    double_reaction_deletion)
with model:
    model.reactions.PGI.knock_out()
    print('PGI knocked out: ', model.optimize())
    solution = model.optimize()
    print('Growth rate: ' + str(solution.objective_value) + ' 1/h')
    df = pandas.DataFrame.from_dict([solution.fluxes]).T
    df.to_csv('FBA_max_biomas1.csv')  

In [3]:
#add heterologous reactions for genes crtE, crtB, crtI
#define pre-existing metabolites
ipdp_c = model.metabolites.get_by_id("ipdp_c")
frdp_c = model.metabolites.get_by_id("frdp_c")
ppi_c = model.metabolites.get_by_id("ppi_c")
nadp_c = model.metabolites.get_by_id("nadp_c")
nadph_c = model.metabolites.get_by_id("nadph_c")
#create new metabolites
ggpp_c = Metabolite('ggpp_c',formula ='C20H36O7P2',name ='Geranylgeranyl pyrophosphate', compartment='c')
phyto_c = Metabolite('phyto_c',formula ='C40H64', name ='Phytoene', compartment ='c')
lyco_c = Metabolite('lyco_c',formula ='C40H56', name ='Lycopene', compartment ='c')
#create new reactions
reaction1 = Reaction('crtE')
reaction1.name = 'geranylgeranyl pyrophosphate synthase (crtE)'
reaction1.subsystem = 'Lycopene Biosynthesis'
reaction1.lower_bound = 0
reaction1.upper_bound = 1000
reaction1.add_metabolites({ipdp_c: -1.0, frdp_c:-1.0,ggpp_c:1.0,ppi_c:1.0})

reaction2 = Reaction('crtB')
reaction2.name = 'phytoene synthase (crtB)'
reaction2.subsystem = 'Lycopene Biosynthesis'
reaction2.lower_bound = 0
reaction2.upper_bound = 1000
reaction2.add_metabolites({ggpp_c:-2.0,phyto_c:1.0,ppi_c:1.0})

reaction3 = Reaction('crtI')
reaction3.name = 'phytoene desaturase (crtI)'
reaction3.subsystem = 'Lycopene Biosynthesis'
reaction3.lower_bound = 0
reaction3.upper_bound = 1000
reaction3.add_metabolites({phyto_c:-1.0,nadp_c:-8.0,lyco_c:1.0,nadph_c:8.0})

reaction4 = Reaction('LYCO-dem')
reaction4.name = 'Lycopene demand'
reaction4.subsystem = 'Lycopene biosynthesis'
reaction4.lower_bound = 0
reaction4.upper_bound = 1000
reaction4.add_metabolites({lyco_c: -1.0})
model.add_reactions([reaction1,reaction2,reaction3,reaction4])

In [ ]:
from pathlib import Path
iJO1366_crtEBI_path = Path()
data_dir = Path(".") / ".." / "Documents" / "GitHub" / "Charlie"
data_dir = data_dir.resolve()
iJO1366_crtEBI_path = data_dir / "iJO1366_crtEBI.xml"
read_sbml_model(str(iJO1366_crtEBI_path.resolve()))


In [15]:
#save edited model
from pathlib import Path
from cobra.io import load_json_model, save_json_model, load_matlab_model, save_matlab_model, read_sbml_model, write_sbml_model
import logging
save_json_model(model,'iJO1366_crtEBI.json')
write_sbml_model(model,'iJO1366_crtEBI.xml')
#load directory
iJO1366_crtEBI_path = 'C:/Users/natha/OneDrive/Documents/GitHub/Charlie/iJO1366_crtEBI.xml'
model = cobra.io.read_sbml_model(str(iJO1366_crtEBI_path))


In [ ]:
#load new model and create new map
builder = Builder(
    map_name='e_coli_core.Core metabolism',
    model_name='iJO1366',
)
builder

In [24]:
#set objective fxns for biomass
model.reactions.get_by_id('BIOMASS_Ec_iJO1366_core_53p95M').objective_coefficient = 0
model.reactions.get_by_id('BIOMASS_Ec_iJO1366_WT_53p95M').objective_coefficient = 1.0
model.reactions.get_by_id('LYCO-dem').objective_coefficient = 0.0
#run FBA
solution = model.optimize()
print('Objective fxn value: ' + str(solution.objective_value))
#export result
df = pandas.DataFrame.from_dict([solution.fluxes]).T
df.to_csv('FBA_max_biomass_crtEBI.csv') 
#solution output
#Output solution
print('Growth Rate (1/h): ' + str(solution.fluxes.get('BIOMASS_Ec_iJO1366_WT_53p95M')))
print('Lycopene Production Rate (mmol/gdcw/h): ' + str(solution.fluxes.get('LYCO-dem')))
print('Lycopene Yield (mol/mol glucose): ' +
str(-solution.fluxes.get('LYCO-dem')/solution.fluxes.get('EX_glc__D_e')))
#conclusion: we see no flux to Lycopene in this instance: the strain is not engineered!

Objective fxn value: 0.986514446952959
Growth Rate (1/h): 0.986514446952959
Lycopene Production Rate (mmol/gdcw/h): 0.0
Lycopene Yield (mol/mol glucose): 0.0


In [25]:
#engineer the strain to produce lycopene
#strategies: #overexpress genes (allow for genes to )
# 2. run a knockout screening to generate strains above a certain growth rate as well as lycopene production

#what are the fluxes in the original solution?
print('Flux in original solution:')
#gene for DXS
print('dxs: ', solution.fluxes.get('DXPS'))
#gene for IDI
print('idi: ', solution.fluxes.get('IPDDI'))
#genes for ispDF
print('ispD ', solution.fluxes.get('MEPCT'))
print('ispF ', solution.fluxes.get('MECDPS'))

Flux in original solution:
dxs:  0.006976630168872916
idi:  0.0009342291812696469
ispD  0.006536644725531895
ispF  0.006536644725531896


In [26]:
from cobra.flux_analysis import flux_variability_analysis
#flux variability analysis
print('Flux ranges in FVA... ')# essentially, what are the expression ranges of the genes to optimize biomass formation?
reactions_OE = [model.reactions.DXPS, model.reactions.IPDDI, model.reactions.MECDPS,
model.reactions.MEPCT]
fva = flux_variability_analysis(model, reaction_list = reactions_OE,
fraction_of_optimum=0.9)
print(fva)


Flux ranges in FVA... 
         minimum   maximum
DXPS    0.006279  0.894814
IPDDI  -0.671443  0.222974
MECDPS  0.005883  0.894418
MEPCT   0.005883  0.894418


In [27]:
#"overexpress" target genes by changing reaction lower bound to maximum value in flux variability analysis
model.reactions.get_by_id('DXPS').lower_bound = 1
model.reactions.get_by_id('IPDDI').lower_bound = 0.5  
model.reactions.get_by_id('MECDPS').lower_bound = 2
model.reactions.get_by_id('MEPCT').lower_bound = 2
#set objective fxns for biomass
model.reactions.get_by_id('BIOMASS_Ec_iJO1366_core_53p95M').objective_coefficient = 0
model.reactions.get_by_id('BIOMASS_Ec_iJO1366_WT_53p95M').objective_coefficient = 1.0
model.reactions.get_by_id('LYCO-dem').objective_coefficient = 0.0
#run FBA
solution = model.optimize()
print('Objective fxn value: ' + str(solution.objective_value))
#export result
df = pandas.DataFrame.from_dict([solution.fluxes]).T
df.to_csv('FBA_max_biomass_crtEBI_overexpressed.csv') 
#solution output
#Output solution
print('Growth Rate (1/h): ' + str(solution.fluxes.get('BIOMASS_Ec_iJO1366_WT_53p95M')))
print('Lycopene Production Rate (mmol/gdcw/h): ' + str(solution.fluxes.get('LYCO-dem')))
print('Lycopene Yield (mol/mol glucose): ' +
str(-solution.fluxes.get('LYCO-dem')/solution.fluxes.get('EX_glc__D_e')))



Objective fxn value: 0.7647818319530075
Growth Rate (1/h): 0.7647818319530075
Lycopene Production Rate (mmol/gdcw/h): 0.24963787580257027
Lycopene Yield (mol/mol glucose): 0.024963787580257028


In [28]:
#visualize overexpression on Escher
builder = Builder(
    map_name='e_coli_core.Core metabolism',
    model_name='iJO1366',
)
builder

Builder()